# Heat Maps in Bokeh

In [1]:
# Note: the below code requires a restart of the notebook the first time it is run
import sys
import os
from traitlets.config.manager import BaseJSONConfigManager
path = os.path.join(sys.prefix, 'etc', 'jupyter', 'nbconfig')
cm = BaseJSONConfigManager(config_dir=path)
cm.update("livereveal", {"scroll": True});

In [2]:
import pandas as pd
import numpy as np
import re
import os
from math import pi
from bokeh.io import show
from bokeh.models import LinearColorMapper, LogColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh import plotting
from bokeh.palettes import Viridis256, magma, RdYlGn, RdYlBu, Plasma256, Viridis256, Greens
plotting.output_notebook()

Loading BokehJS ...

In [3]:
%%time
row_len = 800_000
h5_dir = r'c:\adsb'
h5_file = os.path.join(h5_dir, '2018-06-24.h5')
pickle_name = f'{os.path.basename(h5_file)}-{row_len}.p'
pickle_path = os.path.join(os.getcwd(), 'data', pickle_name)
if os.path.exists(pickle_path):
    df = pd.read_pickle(pickle_path)
else:
    with pd.HDFStore(h5_file) as store:
        columns_to_keep = ['Year', 'Cou', 'Icao', 'Op', 'Type']
        df = store.select('data', stop = row_len, columns=columns_to_keep)
        
    main_ops = ['Southwest', 'American', 'Delta', 'SkyWest', 'Air Canada', 
            'Virgin', 'United','JetBlue', 'Spirit', 'Frontier']
    for o in main_ops:
        df.loc[df.Op.fillna('Other').str.lower().str.contains(o.lower()), 'Op'] = o
    df.loc[~df.Op.isin(main_ops), 'Op'] = 'Other'
    df = df.astype({'Op':'category'})

    df = df.dropna(subset=['Cou', 'Icao'], how='any')
    df.to_pickle(pickle_path)

Wall time: 532 ms


In [4]:
# df.loc[df.Year.isin(['Year','','2104','2105']),'Year'] = np.nan
# cou_vs_year = df.groupby(['Cou', 'Year']).count()['Icao'].unstack().fillna(0)
# cou_vs_year.index.name = 'Country'
# cou_vs_year.columns.name = 'Year'
# cou_vs_year['Total'] = cou_vs_year.sum(axis=1)
# cou_vs_year = cou_vs_year[cou_vs_year.columns[35:]]
# cou_vs_year = cou_vs_year.sort_values('Total', ascending=False)
# del cou_vs_year['Total']
# cou_vs_year = cou_vs_year[:20]
# cou_vs_year_percent = cou_vs_year.div(cou_vs_year.sum(axis=1),axis=0) * 100
# year_vs_cou_percent = cou_vs_year_percent.T
# year_vs_cou = cou_vs_year.T

## Country of Registration vs Purchase Year

In [5]:
column_item_max, row_item_max = 20, 15
col_name = 'Year'
row_name = 'Cou'
row_name_long = 'Year'
col_name_long = 'Country'
title = f"Aircraft Registered to {col_name_long} by {row_name_long}"

top_column_item = list(df[col_name].value_counts()[:column_item_max].index)
row_vs_col = df.groupby([row_name, col_name]).nunique()['Icao'].unstack().fillna(0)
row_vs_col.index.name = row_name
row_vs_col.columns.name = col_name
row_vs_col = row_vs_col[top_column_item]
row_vs_col['Total'] = row_vs_col.sum(axis=1)
row_vs_col = row_vs_col.sort_values('Total', ascending=False)
del row_vs_col['Total']
row_vs_col = row_vs_col[:row_item_max]
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.columns), axis=1)
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.index), axis=0)
# row_vs_col.columns.name = 'Mdl'
row_vs_col_percent = row_vs_col.div(row_vs_col.sum(axis=1),axis=0) * 100
col_vs_row_percent = row_vs_col_percent.T
col_vs_row = row_vs_col.T

col_vs_row_percent.index = col_vs_row_percent.index.astype(str)

c = list(col_vs_row_percent.index)
r = list(col_vs_row_percent.columns)

arr_p = pd.DataFrame(col_vs_row_percent.stack(), columns=['Percentage']).reset_index()
arr_c = pd.DataFrame(col_vs_row.stack(), columns=['Count']).reset_index()
total_by_cou = col_vs_row.sum(axis=0)
arr_c['Total'] = arr_c[row_name].map(total_by_cou)
arr = pd.merge(arr_p, arr_c, how='outer', left_on=[row_name, col_name], right_on=[row_name, col_name])

colors = Plasma256#RdYlBu[11]#, RdYlBu
mapper = LogColorMapper(palette=colors, low=1, high=col_vs_row_percent.max().max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = plotting.figure(title=title,
           x_range=c, y_range=list(reversed(r)),
           x_axis_location="above", plot_width=950, plot_height=900,
           tools=TOOLS, toolbar_location='above',
           tooltips=[(row_name_long, '@'+row_name), 
                     (col_name_long, '@'+col_name), 
                     ('Aircraft Count', '@Count'),
                     ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
                     ("Total Aircraft", '@Total{,}')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x=col_name, y=row_name, width=1, height=1,
       source=arr,
       fill_color={'field': 'Percentage', 'transform': mapper},
       line_color='white')

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')
show(p)

## Country of Registration vs Type/Model

In [6]:
column_item_max, row_item_max = 15, 15
col_name = 'Type'
row_name = 'Cou'
row_name_long = 'Country'
col_name_long = 'Model'
title = f"Aircraft {col_name_long} by {row_name_long}"

top_column_item = list(df[col_name].value_counts()[:column_item_max].index)
row_vs_col = df.groupby([row_name, col_name]).nunique()['Icao'].unstack().fillna(0)
row_vs_col.index.name = row_name
row_vs_col.columns.name = col_name
row_vs_col = row_vs_col[top_column_item]
row_vs_col['Total'] = row_vs_col.sum(axis=1)
row_vs_col = row_vs_col.sort_values('Total', ascending=False)
del row_vs_col['Total']
row_vs_col = row_vs_col[:row_item_max]
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.columns), axis=1)
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.index), axis=0)
row_vs_col_percent = row_vs_col.div(row_vs_col.sum(axis=1),axis=0) * 100
col_vs_row_percent = row_vs_col_percent.T
col_vs_row = row_vs_col.T

col_vs_row_percent.index = col_vs_row_percent.index.astype(str)

c = list(col_vs_row_percent.index)
r = list(col_vs_row_percent.columns)

arr_p = pd.DataFrame(col_vs_row_percent.stack(), columns=['Percentage']).reset_index()
arr_c = pd.DataFrame(col_vs_row.stack(), columns=['Count']).reset_index()
total_by_cou = col_vs_row.sum(axis=0)
arr_c['Total'] = arr_c[row_name].map(total_by_cou)
arr = pd.merge(arr_p, arr_c, how='outer', left_on=[row_name, col_name], right_on=[row_name, col_name])

colors = Plasma256#RdYlBu[11]#, RdYlBu
mapper = LogColorMapper(palette=colors, low=1, high=col_vs_row_percent.max().max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = plotting.figure(title=title,
           x_range=c, y_range=list(reversed(r)),
           x_axis_location="above", plot_width=950, plot_height=900,
           tools=TOOLS, toolbar_location='above',
           tooltips=[(row_name_long, '@'+row_name), 
                     (col_name_long, '@'+col_name), 
                     ('Aircraft Count', '@Count'),
                     ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
                     ("Total Aircraft", '@Total{,}')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x=col_name, y=row_name, width=1, height=1,
       source=arr,
       fill_color={'field': 'Percentage', 'transform': mapper},
       line_color='white')

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')
show(p)

## Aircraft Type/Model vs Operator

In [7]:
column_item_max, row_item_max = 15, 15
col_name = 'Type'
row_name = 'Op'
col_name_long = 'Model'
row_name_long = 'Operator'
title = f"Aircraft {col_name_long} by {row_name_long}"

top_column_item = list(df[col_name].value_counts()[:column_item_max].index)
row_vs_col = df.groupby([row_name, col_name]).nunique()['Icao'].unstack().fillna(0)
row_vs_col.index.name = row_name
row_vs_col.columns.name = col_name
row_vs_col = row_vs_col[top_column_item]
row_vs_col['Total'] = row_vs_col.sum(axis=1)
row_vs_col = row_vs_col.sort_values('Total', ascending=False)
del row_vs_col['Total']
row_vs_col = row_vs_col[:row_item_max]
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.columns), axis=1)
row_vs_col = row_vs_col.reindex(sorted(row_vs_col.index), axis=0)
# row_vs_col.columns.name = 'Mdl'
row_vs_col_percent = row_vs_col.div(row_vs_col.sum(axis=1),axis=0) * 100
col_vs_row_percent = row_vs_col_percent.T
col_vs_row = row_vs_col.T

col_vs_row_percent.index = col_vs_row_percent.index.astype(str)

c = list(col_vs_row_percent.index)
r = list(col_vs_row_percent.columns)

arr_p = pd.DataFrame(col_vs_row_percent.stack(), columns=['Percentage']).reset_index()
arr_c = pd.DataFrame(col_vs_row.stack(), columns=['Count']).reset_index()
total_by_cou = col_vs_row.sum(axis=0)
arr_c['Total'] = arr_c[row_name].map(total_by_cou)
arr = pd.merge(arr_p, arr_c, how='outer', left_on=[row_name, col_name], right_on=[row_name, col_name])

colors = Plasma256#RdYlBu[11]#, RdYlBu
mapper = LogColorMapper(palette=colors, low=1, high=col_vs_row_percent.max().max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = plotting.figure(title=title,
           x_range=c, y_range=list(reversed(r)),
           x_axis_location="above", plot_width=950, plot_height=900,
           tools=TOOLS, toolbar_location='above',
           tooltips=[(row_name_long, '@'+row_name), 
                     (col_name_long, '@'+col_name), 
                     ('Aircraft Count', '@Count'),
                     ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
                     ("Total Aircraft", '@Total{,}')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "12pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x=col_name, y=row_name, width=1, height=1,
       source=arr,
       fill_color={'field': 'Percentage', 'transform': mapper},
       line_color='white')

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
                     ticker=BasicTicker(desired_num_ticks=10),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')
show(p)

In [8]:
# column_item_max, row_item_max = 15, 15
# col_name = 'Type'
# row_name = 'Cou'
# col_name_long = 'Model'
# row_name_long = 'Country'
# title = f"Aircraft Registered to {col_name_long} by {row_name_long}"

# top_column_item = list(df[col_name].value_counts()[:column_item_max].index)
# row_vs_col = df.groupby([row_name, col_name]).nunique()['Icao'].unstack().fillna(0)
# row_vs_col.index.name = row_name
# row_vs_col.columns.name = col_name
# row_vs_col = row_vs_col[top_column_item]
# row_vs_col['Total'] = row_vs_col.sum(axis=1)
# row_vs_col = row_vs_col.sort_values('Total', ascending=False)
# del row_vs_col['Total']
# row_vs_col = row_vs_col[:row_item_max]
# row_vs_col = row_vs_col.reindex(sorted(row_vs_col.columns), axis=1)
# row_vs_col = row_vs_col.reindex(sorted(row_vs_col.index), axis=0)
# # row_vs_col.columns.name = 'Mdl'
# row_vs_col_percent = row_vs_col.div(row_vs_col.sum(axis=1),axis=0) * 100
# col_vs_row_percent = row_vs_col_percent.T
# col_vs_row = row_vs_col.T

# col_vs_row_percent.index = col_vs_row_percent.index.astype(str)

# c = list(col_vs_row_percent.index)
# r = list(col_vs_row_percent.columns)

# arr_p = pd.DataFrame(col_vs_row_percent.stack(), columns=['Percentage']).reset_index()
# arr_c = pd.DataFrame(col_vs_row.stack(), columns=['Count']).reset_index()
# total_by_cou = col_vs_row.sum(axis=0)
# arr_c['Total'] = arr_c[row_name].map(total_by_cou)
# arr = pd.merge(arr_p, arr_c, how='outer', left_on=[row_name, col_name], right_on=[row_name, col_name])

# colors = Plasma256#RdYlBu[11]#, RdYlBu
# mapper = LogColorMapper(palette=colors, low=1, high=col_vs_row_percent.max().max())

# TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

# p = plotting.figure(title=title,
#            x_range=c, y_range=list(reversed(r)),
#            x_axis_location="above", plot_width=950, plot_height=900,
#            tools=TOOLS, toolbar_location='above',
#            tooltips=[(row_name_long, '@'+row_name), 
#                      (col_name_long, '@'+col_name), 
#                      ('Aircraft Count', '@Count'),
#                      ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
#                      ("Total Aircraft", '@Total{,}')])

# p.grid.grid_line_color = None
# p.axis.axis_line_color = None
# p.axis.major_tick_line_color = None
# p.axis.major_label_text_font_size = "12pt"
# p.axis.major_label_standoff = 0
# p.xaxis.major_label_orientation = pi / 3

# p.rect(x=col_name, y=row_name, width=1, height=1,
#        source=arr,
#        fill_color={'field': 'Percentage', 'transform': mapper},
#        line_color='white')

# color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
#                      ticker=BasicTicker(desired_num_ticks=10),
#                      label_standoff=6, border_line_color=None, location=(0, 0))
# p.add_layout(color_bar, 'right')
# show(p)

In [9]:
# model_max, country_max = 15, 15

# top_models = list(df.Type.value_counts()[:model_max].index)
# cou_vs_mdl = df.groupby(['Cou', 'Type']).nunique()['Icao'].unstack().fillna(0)
# cou_vs_mdl.index.name = 'Country'
# cou_vs_mdl.columns.name = 'Mdl'
# cou_vs_mdl = cou_vs_mdl[top_models]
# cou_vs_mdl['Total'] = cou_vs_mdl.sum(axis=1)
# cou_vs_mdl = cou_vs_mdl.sort_values('Total', ascending=False)
# del cou_vs_mdl['Total']
# cou_vs_mdl = cou_vs_mdl[:country_max]
# cou_vs_mdl = cou_vs_mdl.reindex(sorted(cou_vs_mdl.columns), axis=1)
# cou_vs_mdl.columns.name = 'Mdl'
# cou_vs_mdl_percent = cou_vs_mdl.div(cou_vs_mdl.sum(axis=1),axis=0) * 100
# mdl_vs_cou_percent = cou_vs_mdl_percent.T
# mdl_vs_cou = cou_vs_mdl.T

In [10]:
# mdl_vs_cou_percent.index = mdl_vs_cou_percent.index.astype(str)

# mdl = list(mdl_vs_cou_percent.index)
# country = list(mdl_vs_cou_percent.columns)

# arr_p = pd.DataFrame(mdl_vs_cou_percent.stack(), columns=['Percentage']).reset_index()
# arr_c = pd.DataFrame(mdl_vs_cou.stack(), columns=['Count']).reset_index()
# total_by_cou = mdl_vs_cou.sum(axis=0)
# arr_c['Total'] = arr_c.Op.map(total_by_cou)
# arr = pd.merge(arr_p, arr_c, how='outer', left_on=['Op', 'Type'], right_on=['Op', 'Type'])

# colors = Plasma256#RdYlBu[11]#, RdYlBu
# mapper = LogColorMapper(palette=colors, low=1, high=mdl_vs_cou_percent.max().max())

# TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

# p = plotting.figure(title="Aircraft Registered to Country by Model",
#            x_range=mdl, y_range=list(reversed(country)),
#            x_axis_location="above", plot_width=950, plot_height=900,
#            tools=TOOLS, toolbar_location='above',
#            tooltips=[('Operator', '@Op'), 
#                      ('Model', '@Type'), 
#                      ('Aircraft Count', '@Count'),
#                      ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
#                      ("Total Aircraft", '@Total{,}')])

# p.grid.grid_line_color = None
# p.axis.axis_line_color = None
# p.axis.major_tick_line_color = None
# p.axis.major_label_text_font_size = "12pt"
# p.axis.major_label_standoff = 0
# p.xaxis.major_label_orientation = pi / 3

# p.rect(x="Type", y="Op", width=1, height=1,
#        source=arr,
#        fill_color={'field': 'Percentage', 'transform': mapper},
#        line_color='white')

# color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
#                      ticker=BasicTicker(desired_num_ticks=10),
#                      label_standoff=6, border_line_color=None, location=(0, 0))
# p.add_layout(color_bar, 'right')
# show(p)

In [11]:
# model_max, country_max = 15, 15

# top_models = list(df.Op.value_counts()[:model_max].index)
# cou_vs_mdl = df.groupby(['Op', 'Type']).nunique()['Icao'].unstack().fillna(0)
# cou_vs_mdl.index.name = 'Op'
# cou_vs_mdl.columns.name = 'Type'
# # print(cou_vs_mdl.columns)
# cou_vs_mdl = cou_vs_mdl[cou_vs_mdl.index.isin(top_models)]
# cou_vs_mdl['Total'] = cou_vs_mdl.sum(axis=1)
# cou_vs_mdl = cou_vs_mdl.sort_values('Total', ascending=False)
# del cou_vs_mdl['Total']
# top_countries = list(cou_vs_mdl.sum(axis=0).sort_values(ascending=False).index[:country_max])
# cou_vs_mdl = cou_vs_mdl[top_countries]
# cou_vs_mdl = cou_vs_mdl.reindex(sorted(cou_vs_mdl.columns), axis=1)
# cou_vs_mdl = cou_vs_mdl.reindex(sorted(cou_vs_mdl.index), axis=0)
# # cou_vs_mdl.columns.name = 'Type'
# cou_vs_mdl_percent = cou_vs_mdl.div(cou_vs_mdl.sum(axis=1),axis=0) * 100
# mdl_vs_cou_percent = cou_vs_mdl_percent.T
# mdl_vs_cou = cou_vs_mdl.T

In [12]:
# mdl_vs_cou_percent.index = mdl_vs_cou_percent.index.astype(str)

# mdl = list(mdl_vs_cou_percent.index)
# country = list(mdl_vs_cou_percent.columns)

# arr_p = pd.DataFrame(mdl_vs_cou_percent.stack(), columns=['Percentage']).reset_index()
# arr_c = pd.DataFrame(mdl_vs_cou.stack(), columns=['Count']).reset_index()
# total_by_cou = mdl_vs_cou.sum(axis=0)
# arr_c['Total'] = arr_c.Op.map(total_by_cou)
# arr = pd.merge(arr_p, arr_c, how='outer', left_on=['Op', 'Type'], right_on=['Op', 'Type'])



# colors = Plasma256#RdYlBu[11]#, RdYlBu
# mapper = LogColorMapper(palette=colors, low=1, high=mdl_vs_cou_percent.max().max())

# TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

# p = plotting.figure(title="Aircraft Registered to Country by Model",
#            x_range=mdl, y_range=list(reversed(country)),
#            x_axis_location="above", plot_width=950, plot_height=900,
#            tools=TOOLS, toolbar_location='above',
#            tooltips=[('Operator', '@Op'), 
#                      ('Model', '@Type'), 
#                      ('Aircraft Count', '@Count'),
#                      ('Percent of Fleet', '@Percentage{0.0}% of fleet'),
#                      ("Total Aircraft", '@Total{,}')])

# p.grid.grid_line_color = None
# p.axis.axis_line_color = None
# p.axis.major_tick_line_color = None
# p.axis.major_label_text_font_size = "12pt"
# p.axis.major_label_standoff = 0
# p.xaxis.major_label_orientation = pi / 3

# p.rect(x="Type", y="Op", width=1, height=1,
#        source=arr,
#        fill_color={'field': 'Percentage', 'transform': mapper},
#        line_color='white')

# color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="12pt",
#                      ticker=BasicTicker(desired_num_ticks=10),
#                      label_standoff=6, border_line_color=None, location=(0, 0))
# p.add_layout(color_bar, 'right')
# show(p)